In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
# from selenium.common.exceptions import NoSuchElementException as NSEE
# from selenium.common.exceptions import StaleElementReferenceException as SERE
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import numpy as np
import pandas as pd

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get("https://edge.pse.com.ph")
wait = WebDriverWait(driver, 10)
driver.implicitly_wait(10)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.80/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\zddj9\.wdm\drivers\chromedriver\win32\98.0.4758.80]
C:\Users\zddj9\AppData\Local\Temp/ipykernel_8304/3829149139.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.find_element(By.XPATH,"//a[contains(@href,'/companyDirectory/form.do')]").click()

In [4]:
elements = driver.find_element(By.XPATH,"//*[@id='dataList']")
# element = wait.until(EC.visibility_of_element_located((By.XPATH, "//*[@id='dataList']")))
print(type(driver))


<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [5]:
driver.implicitly_wait(10)
unclean_company_id_stocks = elements.find_elements(By.XPATH,".//tr")

unclean_company_info_list = []
print(type(unclean_company_id_stocks))

for unclean_line in unclean_company_id_stocks:
    unclean_company_info_list.append(unclean_line.get_attribute('innerHTML')
                        .replace(' ','')
                        .replace("'","")
                        .strip()
                        .split('\n'))
    print(unclean_line.get_attribute('innerHTML'))


<class 'list'>

    <th>Company Name</th>
    <th>Stock Symbol</th>
    <th>Sector</th>
    <th>Subsector</th>
    <th>Listing Date</th>
  

      <td><a href="#company" onclick="cmDetail('29','146');return false;">2GO Group, Inc.</a></td>
      <td class="alignC"><a href="#company" onclick="cmDetail('29','146');return false;">2GO</a></td>
      <td>Services</td>
      <td>Transportation Services</td>
      <td class="alignC">May 15, 1995</td>
    

      <td><a href="#company" onclick="cmDetail('55','347');return false;">Asia Amalgamated Holdings Corporation</a></td>
      <td class="alignC"><a href="#company" onclick="cmDetail('55','347');return false;">AAA</a></td>
      <td>Holding Firms</td>
      <td>Holding Firms</td>
      <td class="alignC">Mar 22, 1973</td>
    

      <td><a href="#company" onclick="cmDetail('19','181');return false;">Atok-Big Wedge Co., Inc.</a></td>
      <td class="alignC"><a href="#company" onclick="cmDetail('19','181');return false;">AB</a></td>
      <

In [6]:
unclean_company_info_list

[['<th>CompanyName</th>',
  '<th>StockSymbol</th>',
  '<th>Sector</th>',
  '<th>Subsector</th>',
  '<th>ListingDate</th>'],
 ['<td><ahref="#company"onclick="cmDetail(29,146);returnfalse;">2GOGroup,Inc.</a></td>',
  '<tdclass="alignC"><ahref="#company"onclick="cmDetail(29,146);returnfalse;">2GO</a></td>',
  '<td>Services</td>',
  '<td>TransportationServices</td>',
  '<tdclass="alignC">May15,1995</td>'],
 ['<td><ahref="#company"onclick="cmDetail(55,347);returnfalse;">AsiaAmalgamatedHoldingsCorporation</a></td>',
  '<tdclass="alignC"><ahref="#company"onclick="cmDetail(55,347);returnfalse;">AAA</a></td>',
  '<td>HoldingFirms</td>',
  '<td>HoldingFirms</td>',
  '<tdclass="alignC">Mar22,1973</td>'],
 ['<td><ahref="#company"onclick="cmDetail(19,181);returnfalse;">Atok-BigWedgeCo.,Inc.</a></td>',
  '<tdclass="alignC"><ahref="#company"onclick="cmDetail(19,181);returnfalse;">AB</a></td>',
  '<td>MiningandOil</td>',
  '<td>Mining</td>',
  '<tdclass="alignC">Jan08,1948</td>'],
 ['<td><ahref="#comp

In [7]:
clean_company_info_list = unclean_company_info_list[1:]
clean_company_info_list

[['<td><ahref="#company"onclick="cmDetail(29,146);returnfalse;">2GOGroup,Inc.</a></td>',
  '<tdclass="alignC"><ahref="#company"onclick="cmDetail(29,146);returnfalse;">2GO</a></td>',
  '<td>Services</td>',
  '<td>TransportationServices</td>',
  '<tdclass="alignC">May15,1995</td>'],
 ['<td><ahref="#company"onclick="cmDetail(55,347);returnfalse;">AsiaAmalgamatedHoldingsCorporation</a></td>',
  '<tdclass="alignC"><ahref="#company"onclick="cmDetail(55,347);returnfalse;">AAA</a></td>',
  '<td>HoldingFirms</td>',
  '<td>HoldingFirms</td>',
  '<tdclass="alignC">Mar22,1973</td>'],
 ['<td><ahref="#company"onclick="cmDetail(19,181);returnfalse;">Atok-BigWedgeCo.,Inc.</a></td>',
  '<tdclass="alignC"><ahref="#company"onclick="cmDetail(19,181);returnfalse;">AB</a></td>',
  '<td>MiningandOil</td>',
  '<td>Mining</td>',
  '<tdclass="alignC">Jan08,1948</td>'],
 ['<td><ahref="#company"onclick="cmDetail(174,173);returnfalse;">AbaCoreCapitalHoldings,Inc.</a></td>',
  '<tdclass="alignC"><ahref="#company"on

In [9]:
company_id_regex = re.compile('(?<=\()([0-9]+)')
company_symbol_regex = re.compile("(?<=;\">)(\S+)(?=</a)")
company_sector_regex = re.compile('(?<=\>)(\w+)')
company_subsector_regex = re.compile('(?<=\>)(\w+)')

company_id = []
company_symbol = []
company_sector = []
company_subsector = []

for company in clean_company_info_list:
    company_id.append(company_id_regex.search(company[1]).group())
    company_symbol.append(company_symbol_regex.search(company[1]).group())
    company_sector.append(company_sector_regex.search(company[2]).group())   
    company_subsector.append(company_subsector_regex.search(company[3]).group())

company_df = pd.DataFrame()

company_df['Company ID'] = np.array(company_id)
company_df['Stock Symbol'] = np.array(company_symbol)
company_df['Sector'] = np.array(company_sector)
company_df['Sub-sector'] = np.array(company_subsector)


In [21]:
company_df

,Company ID,Stock Symbol,Sector,Sub-sector
0,29,2GO,Services,TransportationServices
1,55,AAA,HoldingFirms,HoldingFirms
2,19,AB,MiningandOil,Mining
3,174,ABA,HoldingFirms,HoldingFirms
4,176,ABG,HoldingFirms,HoldingFirms
5,114,ABS,Services,Media
6,15,ABSP,Services,Media
7,57,AC,HoldingFirms,HoldingFirms
8,48,ACE,Services,Hotel
9,233,ACEN,Industrial,Electricity


Company ID is for constructing the company information link e.g. https://edge.pse.com.ph/companyPage/stockData.do?cmpy_id=29 for 2GO